# Set up for the Environment (Local/ Google Colab)

In [ ]:
try: 
    from google.colab import drive
    drive.mount('/content/drive')
    in_google_colab = True

    print("The following versions are required: Tensorfow < 2.0.0 and Keras == 2.2.4.")
    print("There might be a few restarts in this section if you're running on Google Colab")
except:
    in_google_colab = False

In [ ]:
# Checking Tensorflow Version
if in_google_colab == True:
    %tensorflow_version 1.x

import tensorflow as tf
if in_google_colab == True and int(tf.__version__.split("-")[0][0]) == 2:
    print("We need the Tensorflow version to be < 2.0.0 for compatibility issues")

    print("Do reset the runtime after we close it")
    import os
    os.kill(os.getpid(), 9)
print('Tensorflow version is {}'.format(tf.__version__))

In [ ]:
# Checking if there is a GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# Checking Keras Version
import keras
keras_version = keras.__version__
print('Keras version is {}'.format(keras_version))

if keras_version != '2.2.4':
    print("We need the Keras version to be 2.2.4 for compatibility issues")

    print("Downloading and installing Keras 2.2.4")
    print("Do reset the runtime after we close it")
    !pip install q keras==2.2.4
    
    import os
    os.kill(os.getpid(), 9)

else:
    print("Keras version is 2.2.4, as required.")

In [ ]:
if in_google_colab:
    print("Currently in google colab, unpacking the files")
    !rm -r /content/mask_rcnn_envir
    !unzip /content/drive/'My Drive'/colab_notebooks/mask_rcnn_envir.zip  -d /content
    %cd mask_rcnn_envir/
else:
    print("Not in google colab")
    
    import os
    if os.path.isdir("./mask_rcnn_envir") is False:
        print("Folder is still zipped, unzipping it now.")
        import zipfile
        with zipfile.ZipFile("./mask_rcnn_envir.zip", "r") as zip_ref:
            zip_ref.extractall()
    else: 
        print("Folder is already unzipped.")
    %cd mask_rcnn_envir/
    
    # Seems this piece of code is required, if not my GPU will freeze
    def get_session():
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        return tf.Session(config=config)

    keras.backend.tensorflow_backend.set_session(get_session())

# Import the required libraries

In [ ]:
from mask_rcnn_v2 import Config, MaskRCNN, buildMASKRCNN, Dataset, CocoLikeDataset, display_top_masks, display_instances, load_image_gt, mold_image, compute_ap, log
import numpy as np
from numpy import zeros
from numpy import asarray
import colorsys
import argparse
import imutils
import random
import cv2
import os
import time
import json
from PIL import Image, ImageDraw
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from keras.models import load_model

#from os import listdir
#from xml.etree import ElementTree

#from pycocotools import mask as maskUtils

from imgaug import augmenters as iaa

import pandas as pd

%matplotlib inline

# Set up the config/ load the dataset

In [ ]:
class myMaskRCNNConfig(Config):
    NAME = 'MaskRCNN_config'
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 3 + 1
    
    STEPS_PER_EPOCH = 131

    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512     

    BACKBONE = 'resnet101'                       

    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256) 
    
    RPN_NMS_THRESHOLD = 0.8

    DETECTION_MIN_CONFIDENCE = 0.7

In [ ]:
config = myMaskRCNNConfig()

In [ ]:
config.display()

In [ ]:
dataset_train = CocoLikeDataset()
dataset_train.load_data('./annotations/instances_openimagev5tococo_train2020.json', './openimagev5tococo_train2020')
dataset_train.prepare()

In [ ]:
dataset_val = CocoLikeDataset()
dataset_val.load_data('./annotations/instances_openimagev5tococo_val2020.json', './openimagev5tococo_val2020')
dataset_val.prepare()

In [ ]:
dataset_train.class_names

In [ ]:
dataset_val.class_names

In [ ]:
dataset = dataset_train
image_ids = np.random.choice(dataset.image_ids, 4)
for image_id in image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    display_top_masks(image, mask, class_ids, dataset.class_names)

# Do Image Augmentation on dataset

In [ ]:
augmentation = iaa.Sequential([
    #geometric transform
    iaa.OneOf([ 
        iaa.Affine(
            scale={'x': (0.98, 1.02), 'y': (0.98, 1.02)},
            translate_percent={'x': (-0.05, 0.05), 'y': (-0.05, 0.05)}, #{'x': (-0.02, 0.02), 'y': (-0.04, 0.04)},
            rotate=(-10, 10), #(-2, 2),
            shear=(-5, 5), #(-1, 1),
        ),
        iaa.PiecewiseAffine(scale=(0.001, 0.025)),
    ]),

    #flip images left-right
    iaa.Fliplr(0.35),
    
    #brightness or contrast
    iaa.OneOf([
        iaa.Multiply((0.8, 1.2)), #(0.9, 1.1)),
        iaa.ContrastNormalization((0.8, 1.2)), #(0.9, 1.1)),
    ]),
    
    #blur or sharpen
    iaa.OneOf([
        iaa.GaussianBlur(sigma=(0.0, 0.25)),
        iaa.Sharpen(alpha=(0.0, 0.25)),
    ]),
])

In [ ]:
#test on the last image of the loop
imggrid = augmentation.draw_grid(image[:,:,0], cols=5, rows=2)
plt.figure(figsize=(30, 12))
_ = plt.imshow(imggrid[:,:,0], cmap='gray')

# Do Transfer Learning  with Mask RCNN

In [ ]:
model = buildMASKRCNN(config=config, model_dir="./")

#exclude the last few layers from training for ResNet101
#exclude the last layers is to match the number of classes in the new dataset
model.load_weights('./mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
model.keras_model.summary()

In [ ]:
model.fit(dataset_train, dataset_val, 
            epochs=40, 
            learning_rate=config.LEARNING_RATE, 
            layers='heads',
            augmentation=augmentation
            )

In [ ]:
history = model.keras_model.history.history

In [ ]:
# Fine Tune ALL layers
model.fit(dataset_train, dataset_val, 
            epochs=60, 
            learning_rate=config.LEARNING_RATE, 
            layers='all',
            augmentation=augmentation
            )

In [ ]:
new_history = model.keras_model.history.history
for k in new_history:
    history[k] = history[k] + new_history[k]

In [ ]:
model.fit(dataset_train, dataset_val, 
            epochs=80, 
            learning_rate=config.LEARNING_RATE * 1e-1, 
            layers='all',
            augmentation=augmentation
            )

In [ ]:
new_history = model.keras_model.history.history
for k in new_history:
    history[k] = history[k] + new_history[k]

In [ ]:
model.fit(dataset_train, dataset_val, 
            epochs=100, 
            learning_rate=config.LEARNING_RATE * 1e-2, 
            layers='all',
            augmentation=augmentation
            )

In [ ]:
new_history = model.keras_model.history.history
for k in new_history:
    history[k] = history[k] + new_history[k]

In [ ]:
model.fit(dataset_train, dataset_val, 
            epochs=120, 
            learning_rate=config.LEARNING_RATE * 1e-3, 
            layers='all',
            augmentation=augmentation
            )

In [ ]:
new_history = model.keras_model.history.history
for k in new_history:
    history[k] = history[k] + new_history[k]

In [ ]:
epochs = range(1, len(next(iter(history.values())))+1)
pd.DataFrame(history, index=epochs)

In [ ]:
df = pd.DataFrame(history, index=epochs)
df.to_csv('data.csv')
!cp data.csv "/content/drive/My Drive/"

In [ ]:
plt.figure(figsize=(17,5))

plt.subplot(131)
plt.plot(epochs, history["loss"], label="Train loss")
plt.plot(epochs, history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(132)
plt.plot(epochs, history["mrcnn_class_loss"], label="Train class loss")
plt.plot(epochs, history["val_mrcnn_class_loss"], label="Valid class loss")
plt.legend()
plt.subplot(133)
plt.plot(epochs, history["mrcnn_bbox_loss"], label="Train Bounding Box loss")
plt.plot(epochs, history["val_mrcnn_bbox_loss"], label="Valid Bounding Box loss")
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(17,5))

plt.subplot(131)
plt.plot(epochs, history["mrcnn_mask_loss"], label="Train Mask loss")
plt.plot(epochs, history["val_mrcnn_mask_loss"], label="Valid Mask loss")
plt.legend()
plt.subplot(132)
plt.plot(epochs, history["rpn_bbox_loss"], label="Train PRN box loss")
plt.plot(epochs, history["val_rpn_bbox_loss"], label="Valid RPN box loss")
plt.legend()
plt.subplot(133)
plt.plot(epochs, history["rpn_class_loss"], label="Train RPN class loss")
plt.plot(epochs, history["val_rpn_class_loss"], label="Valid PRN class loss")
plt.legend()

plt.show()

In [ ]:
#to save the best model
if in_google_colab:
    import glob
    extract_weights = sorted(glob.glob("/content/mask_rcnn_envir/maskrcnn_config*/*"))[-1]

    #!mv {extract_weights} ./custom_mask_rcnn_weights.h5 #Uncomment to save the custom trained Mask RCNN weights
    #!mv {extract_weights} /content/drive/'My Drive'/custom_mask_rcnn_weights.h5 #Uncomment to save the custom trained Mask RCNN weights

In [ ]:
# To keep it alive in colab if you're AFK
"""
# Ctrl + Shift + i
# Does a button click every every 5mins, copy this into your developer console

function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}
setInterval(ClickConnect, 300000) 
"""
print()

# Do evaluation on Validation Dataset

In [ ]:
class myMaskRCNNInferenceConfig(Config):
    NAME = 'MaskRCNN_inference_config'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = myMaskRCNNConfig()

In [ ]:
model_path = './custom_mask_rcnn_weights.h5'

In [ ]:
model_inference = MaskRCNN(mode='inference', model_dir='./', config=inference_config)

model_inference.load_weights(model_path, by_name=True)

class_names = ['BG', 'Dress', 'Suit', 'Jeans']

In [ ]:
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset_val, inference_config, image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

# Display the Ground Truth Mask on the Image
display_instances(original_image, gt_bbox, gt_mask, gt_class_id, dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model_inference.predict([original_image], verbose=1)

r = results[0]

# Display the Predicted Mask on the Image
display_instances(original_image, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], figsize=(8, 8))

In [ ]:
image_ids = dataset_val.image_ids
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset_val, inference_config, image_id, use_mini_mask=False)
    molded_images = np.expand_dims(mold_image(image, inference_config), 0)
    # Run object detection
    results = model_inference.predict([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

# Detection on Scoring Dataset

In [ ]:
scoring_base_filepath = './test/'

In [ ]:
test_image1 = cv2.cvtColor(cv2.imread(scoring_base_filepath + 'suit1.jpg'), cv2.COLOR_BGR2RGB)
results = model_inference.predict([test_image1], verbose=1)

r = results[0]
display_instances(test_image1, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], figsize=(8, 8))

In [ ]:
test_image2 = cv2.cvtColor(cv2.imread(scoring_base_filepath + 'suit2.jpg'), cv2.COLOR_BGR2RGB)
results = model_inference.predict([test_image2], verbose=1)

r = results[0]
display_instances(test_image2, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], figsize=(8, 8))

In [ ]:
test_image3 = cv2.cvtColor(cv2.imread(scoring_base_filepath + 'dress1.jpg'), cv2.COLOR_BGR2RGB)
results = model_inference.predict([test_image3], verbose=1)

r = results[0]
display_instances(test_image3, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], figsize=(8, 8))

In [ ]:
test_image4 = cv2.cvtColor(cv2.imread(scoring_base_filepath + 'dress2.jpg'), cv2.COLOR_BGR2RGB)
results = model_inference.predict([test_image4], verbose=1)

r = results[0]
display_instances(test_image4, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], figsize=(8, 8))

In [ ]:
test_image5 = cv2.cvtColor(cv2.imread(scoring_base_filepath + 'jeans.jpg'), cv2.COLOR_BGR2RGB)
results = model_inference.predict([test_image5], verbose=1)

r = results[0]
display_instances(test_image5, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], figsize=(8, 8))